In [ ]:
from langchain_google_vertexai import ChatVertexAI
from langchain_core.prompts import PromptTemplate

llm = ChatVertexAI(model="gemini-2.0-flash-001")

# ToolNode

We will re-use tools we created before:

In [ ]:
import math
import numexpr as ne
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun


search = DuckDuckGoSearchRun()

@tool
def calculator(expression: str) -> str:
    """Calculates a single mathematical expression, incl. complex numbers.

    Always add * to operations, examples:
      73i -> 73*i
      7pi**2 -> 7*pi**2
    """
    math_constants = {"pi": math.pi, "i": 1j, "e": math.exp}
    result = ne.evaluate(expression.strip(), local_dict=math_constants)
    return str(result)


llm_with_tools = ChatVertexAI(model="gemini-2.0-flash-001").bind_tools([search, calculator])

We will build our own ReACT agent, but this time we will use a `ToolNode` to execute tool calls automatically:

In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import MessagesState, StateGraph, START, END

def invoke_llm(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

builder = StateGraph(MessagesState)
builder.add_node("invoke_llm", invoke_llm)
builder.add_node("tools", ToolNode([search, calculator]))

builder.add_edge(START, "invoke_llm")
builder.add_conditional_edges("invoke_llm", tools_condition)
builder.add_edge("tools", "invoke_llm")
builder.add_edge("tools", END)
builder.add_edge("invoke_llm", END)
graph = builder.compile()

In [ ]:
for e in graph.stream({"messages": ("human", "How much is 2+2")}):
  print(e)

{'invoke_llm': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'calculator', 'arguments': '{"expression": "2+2"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 95, 'candidates_token_count': 5, 'total_token_count': 100, 'cached_content_token_count': 0}, 'finish_reason': 'STOP', 'avg_logprobs': -4.0858518332242966e-05}, id='run-56f3fb75-3c82-4e03-84fe-b5d6cd1b5389-0', tool_calls=[{'name': 'calculator', 'args': {'expression': '2+2'}, 'id': '72c9ac3c-6057-4953-b958-95cc3df1d6e1', 'type': 'tool_call'}], usage_metadata={'input_tokens': 95, 'output_tokens': 5, 'total_tokens': 100})]}}
{'tools': {'messages': [ToolMessage(content='4', name='calculator', id='74c47f98-cc52-424b-90b2-b5fa3f398632', tool_call_id='72c9ac3c-6057-4953-b958-95cc3df1d6e1')]}}
{'invoke_llm': {'messages': [AIMessage(content='2+2 is 4.\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_me

Now we will look at examples how to use custom tools to easily extract dates from user input:

In [ ]:
examples = [
  "I signed my contract 2 years ago",
  "I started the deal with your company in February last year",
  "Our contract started on March 24th two years ago"
]

In [ ]:
from datetime import date, timedelta


@tool
def get_date(year: int, month: int = 1, day: int = 1) -> date:
    """Returns a date object given year, month and day.

      Default month and day are 1 (January) and 1.
      Examples in YYYY-MM-DD format:
        2023-07-27 -> date(2023, 7, 27)
        2022-12-15 -> date(2022, 12, 15)
        March 2022 -> date(2022, 3)
        2021 -> date(2021)
    """
    return date(year, month, day).isoformat()


@tool
def time_difference(days: int = 0, weeks: int = 0, months: int = 0, years: int = 0) -> date:
    """Returns a date given a difference in days, weeks, months and years relative to the current date.

    By default, dayss, weeks, months and years are 0.
    Examples:
      two weeks ago -> time_difference(weeks=2)
      last year -> time_difference(years=1)
    """
    dt = date.today() - timedelta(days=days, weeks=weeks)
    new_year = dt.year+(dt.month-months) // 12 - years
    new_month = (dt.month-months) % 12
    return dt.replace(year=new_year, month=new_month)

In [ ]:
from langgraph.prebuilt import create_react_agent


llm = ChatVertexAI(model="gemini-1.5-pro-002")

agent = create_react_agent(
    llm, [get_date, time_difference], prompt="Extract the starting date of a contract. Current year is 2025.")


for example in examples:
  result = agent.invoke({"messages": [("user", example)]})
  print(example, result["messages"][-1].content)

I signed my contract 2 years ago The contract started on 2023-02-07.

I started the deal with your company in February last year The contract started on 2024-02-01.

Our contract started on March 24th two years ago The contract started on 2023-03-24.



In [ ]:
for e in agent.stream({"messages": [("user", examples[1])]}):
  print(e)

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_date', 'arguments': '{"year": 2022.0, "month": 2.0}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 228, 'candidates_token_count': 5, 'total_token_count': 233, 'cached_content_token_count': 0}, 'finish_reason': 'STOP', 'avg_logprobs': -0.09780421853065491}, id='run-ee39f3f6-d94c-4199-98ff-753e81db5742-0', tool_calls=[{'name': 'get_date', 'args': {'year': 2022.0, 'month': 2.0}, 'id': 'b78ff7eb-bd4f-4522-9664-0aa738944f1a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 228, 'output_tokens': 5, 'total_tokens': 233})]}}
{'tools': {'messages': [ToolMessage(content='2022-02-01', name='get_date', id='13f8cdad-bb85-4582-83c5-6ea2020d2689', tool_call_id='b78ff7eb-bd4f-4522-9664-0aa738944f1a')]}}
{'agent': {'messages': [AIMessage(content='The contract started on 2022-02-01.\n', additional_kwargs={}, response_metadata={'is_blocked': Fa

We can also explore the answer the model produced to pass to a user:

In [ ]:
result["messages"][-1].content

'OK. The contract started on 2027-02-07.\n'